In [1]:
import pandas as pd
import numpy as np
import scipy
from scipy import stats
from numpy import sqrt
import matplotlib.pyplot as plt
from scipy.stats import shapiro
from scipy.stats import mannwhitneyu
from statsmodels.stats.multitest import multipletests

In [2]:
som=pd.read_excel(r"C:\Users\Client\OneDrive - Queen Mary, University of London\PhD Shared folder\SomaScan Results\Soma results.xlsx", header=0, index_col="Gene name")
som.head()

,HC 1,HC 2,ALS S1,ALS S2,ALS F1,ALS S3,HC 3,HC 4,ALS S4,HC 5,...,ALS S21,ALS F22,HC 25,ALS F23,ALS S22,ALS F24,ALS S23,HC 6,ALS F25,ALS S24
Gene name,,,,,,,,,,,,,,,,,,,,,
NEFL,2269.1,221.6,343.8,301.1,283.6,302.9,233.6,214.9,253.0,529.3,...,272.8,274.0,347.4,404.1,238.5,323.0,349.1,312.1,232.4,233.2
CHIT1,1400.3,1555.5,2055.3,1346.7,971.4,2752.7,1346.1,736.7,1421.0,3755.3,...,1092.5,5255.4,1033.5,297.2,952.0,901.5,894.7,1719.8,2007.7,1041.7
MPZ,1781.2,1039.1,1174.6,974.8,1031.7,932.6,1959.6,1217.4,1027.9,707.9,...,786.7,894.6,1068.8,1049.1,1264.8,1094.9,1408.4,808.8,956.3,1408.9
GRIA4,3457.8,3768.6,2770.5,3439.6,2608.8,2955.4,2169.2,3565.3,2232.5,1992.9,...,2943.5,3397.9,2309.0,2927.2,3282.2,2893.5,4667.4,2459.0,2485.6,5034.1
PLD3,418.4,750.4,484.0,1208.1,1159.7,490.1,583.7,283.9,432.0,439.8,...,435.7,627.8,535.0,453.4,648.4,652.9,507.6,544.4,660.3,595.6


In [3]:
# Apply log2 transform
soma = som.applymap(lambda x: np.log2(x + 1))  # handles zeros
soma.head()

,HC 1,HC 2,ALS S1,ALS S2,ALS F1,ALS S3,HC 3,HC 4,ALS S4,HC 5,...,ALS S21,ALS F22,HC 25,ALS F23,ALS S22,ALS F24,ALS S23,HC 6,ALS F25,ALS S24
Gene name,,,,,,,,,,,,,,,,,,,,,
NEFL,11.148540,7.798310,8.429616,8.238882,8.152792,8.247453,7.874059,7.754219,7.988685,9.050665,...,8.096979,8.103288,8.444601,8.662134,7.903882,8.339850,8.451623,8.290480,7.866661,7.871597
CHIT1,10.452550,10.604090,11.005835,10.396284,9.925406,11.427156,10.395641,9.526890,10.473706,11.875097,...,10.094738,12.359859,10.014718,8.220136,9.896332,9.817783,9.806872,10.748864,10.972046,10.026108
MPZ,10.799444,10.022507,10.199182,9.930442,10.012205,9.866661,10.937080,10.250772,10.006887,9.469438,...,9.621502,9.806711,10.063125,10.036311,10.305834,10.097900,10.460865,9.661422,9.902827,10.461377
GRIA4,11.756056,11.880196,11.436451,11.748444,11.349724,11.529626,11.083612,11.800212,11.125091,10.961377,...,11.523807,11.730852,11.173677,11.515798,11.680887,11.499098,12.188712,11.264443,11.279959,12.297805
PLD3,8.712183,9.553437,8.921841,10.239718,10.180779,8.939873,9.191553,8.154312,8.758223,8.783980,...,8.770499,9.296457,9.066089,8.827819,9.342964,9.352926,8.990388,9.091171,9.369161,9.220620


In [4]:
stat, p_value = shapiro(soma)

if p_value > 0.05:
    print("Data is normally distributed (fail to reject H0)")
else:
    print("Data is not normally distributed (reject H0)")

Data is not normally distributed (reject H0)


C:\Users\Client\anaconda3\lib\site-packages\scipy\stats\_morestats.py:1800: UserWarning: p-value may not be accurate for N > 5000.
  warnings.warn("p-value may not be accurate for N > 5000.")


In [5]:
# Automatically identify group columns by their prefixes
group1= soma.loc[:, soma.columns.str.startswith('HC ')]
group2= soma.loc[:, soma.columns.str.startswith('ALS ')]

In [6]:
stat, p_value = mannwhitneyu(group1.iloc[0], group2.iloc[0], alternative='two-sided')

stats = []
p_vals = []
for i in range(group1.shape[0]):
    stat, p_value = mannwhitneyu(group1.iloc[i], group2.iloc[i], alternative='two-sided', method ="exact")
    stats.append(stat)
    p_vals.append(p_value)

In [7]:
# Add the results DataFrame to the original data DataFrame
soma['U-stat'] = stats
soma['p_value'] = p_vals
soma.head()

,HC 1,HC 2,ALS S1,ALS S2,ALS F1,ALS S3,HC 3,HC 4,ALS S4,HC 5,...,HC 25,ALS F23,ALS S22,ALS F24,ALS S23,HC 6,ALS F25,ALS S24,U-stat,p_value
Gene name,,,,,,,,,,,,,,,,,,,,,
NEFL,11.148540,7.798310,8.429616,8.238882,8.152792,8.247453,7.874059,7.754219,7.988685,9.050665,...,8.444601,8.662134,7.903882,8.339850,8.451623,8.290480,7.866661,7.871597,487.5,0.199409
CHIT1,10.452550,10.604090,11.005835,10.396284,9.925406,11.427156,10.395641,9.526890,10.473706,11.875097,...,10.014718,8.220136,9.896332,9.817783,9.806872,10.748864,10.972046,10.026108,545.0,0.531757
MPZ,10.799444,10.022507,10.199182,9.930442,10.012205,9.866661,10.937080,10.250772,10.006887,9.469438,...,10.063125,10.036311,10.305834,10.097900,10.460865,9.661422,9.902827,10.461377,676.0,0.385924
GRIA4,11.756056,11.880196,11.436451,11.748444,11.349724,11.529626,11.083612,11.800212,11.125091,10.961377,...,11.173677,11.515798,11.680887,11.499098,12.188712,11.264443,11.279959,12.297805,595.0,0.958860
PLD3,8.712183,9.553437,8.921841,10.239718,10.180779,8.939873,9.191553,8.154312,8.758223,8.783980,...,9.066089,8.827819,9.342964,9.352926,8.990388,9.091171,9.369161,9.220620,642.0,0.634091


In [8]:
# Apply Benjamini-Hochberg correction (optional)
_, p_adjusted, _, _ = multipletests(soma['p_value'], method='fdr_bh')
soma['p_adjusted_bh'] = p_adjusted
soma.head()

,HC 1,HC 2,ALS S1,ALS S2,ALS F1,ALS S3,HC 3,HC 4,ALS S4,HC 5,...,ALS F23,ALS S22,ALS F24,ALS S23,HC 6,ALS F25,ALS S24,U-stat,p_value,p_adjusted_bh
Gene name,,,,,,,,,,,,,,,,,,,,,
NEFL,11.148540,7.798310,8.429616,8.238882,8.152792,8.247453,7.874059,7.754219,7.988685,9.050665,...,8.662134,7.903882,8.339850,8.451623,8.290480,7.866661,7.871597,487.5,0.199409,0.463742
CHIT1,10.452550,10.604090,11.005835,10.396284,9.925406,11.427156,10.395641,9.526890,10.473706,11.875097,...,8.220136,9.896332,9.817783,9.806872,10.748864,10.972046,10.026108,545.0,0.531757,0.690594
MPZ,10.799444,10.022507,10.199182,9.930442,10.012205,9.866661,10.937080,10.250772,10.006887,9.469438,...,10.036311,10.305834,10.097900,10.460865,9.661422,9.902827,10.461377,676.0,0.385924,0.576006
GRIA4,11.756056,11.880196,11.436451,11.748444,11.349724,11.529626,11.083612,11.800212,11.125091,10.961377,...,11.515798,11.680887,11.499098,12.188712,11.264443,11.279959,12.297805,595.0,0.958860,0.958860
PLD3,8.712183,9.553437,8.921841,10.239718,10.180779,8.939873,9.191553,8.154312,8.758223,8.783980,...,8.827819,9.342964,9.352926,8.990388,9.091171,9.369161,9.220620,642.0,0.634091,0.763965


In [9]:
#df['average'] = df.mean(numeric_only=True, axis=1)
soma['Mean Control'] = soma.loc[:, soma.columns.str.startswith('HC ')].mean(axis=1)
soma['Mean ALS'] = soma.loc[:, soma.columns.str.startswith('ALS ')].mean(axis=1)

In [10]:
#Fold Change
soma["FC ALS-C"]= soma["Mean ALS"]/soma["Mean Control"]

In [11]:
#Log 2 FC
soma["Log2FC ALS-C"]=np.log2(soma["FC ALS-C"])
soma.head()

,HC 1,HC 2,ALS S1,ALS S2,ALS F1,ALS S3,HC 3,HC 4,ALS S4,HC 5,...,HC 6,ALS F25,ALS S24,U-stat,p_value,p_adjusted_bh,Mean Control,Mean ALS,FC ALS-C,Log2FC ALS-C
Gene name,,,,,,,,,,,,,,,,,,,,,
NEFL,11.148540,7.798310,8.429616,8.238882,8.152792,8.247453,7.874059,7.754219,7.988685,9.050665,...,8.290480,7.866661,7.871597,487.5,0.199409,0.463742,8.311382,8.304977,0.999229,-0.001112
CHIT1,10.452550,10.604090,11.005835,10.396284,9.925406,11.427156,10.395641,9.526890,10.473706,11.875097,...,10.748864,10.972046,10.026108,545.0,0.531757,0.690594,10.236831,10.321109,1.008233,0.011829
MPZ,10.799444,10.022507,10.199182,9.930442,10.012205,9.866661,10.937080,10.250772,10.006887,9.469438,...,9.661422,9.902827,10.461377,676.0,0.385924,0.576006,10.200432,10.144128,0.994480,-0.007985
GRIA4,11.756056,11.880196,11.436451,11.748444,11.349724,11.529626,11.083612,11.800212,11.125091,10.961377,...,11.264443,11.279959,12.297805,595.0,0.958860,0.958860,11.568672,11.619487,1.004392,0.006323
PLD3,8.712183,9.553437,8.921841,10.239718,10.180779,8.939873,9.191553,8.154312,8.758223,8.783980,...,9.091171,9.369161,9.220620,642.0,0.634091,0.763965,9.279881,9.201568,0.991561,-0.012227


In [12]:
soma.to_excel(r"C:\Users\Client\OneDrive - Queen Mary, University of London\PhD Shared folder\SomaScan Results\Soma Mann-W no mistake.xlsx")